# Chemical Process Anomaly Detection Baseline 🕵️
### 화학 공정 데이터를 분석하고 🧐, 이상 탐지 Baseline Model 을 학습합니다 📖.


## Table of Contents
1. 필요 Library Import
2. Load Data
3. EDA
4. Data Process
5. Model Training
6. Model Inference


### 1. 필요 Library Import

- 필요 Library 설치를 위해 아래 명령어를 실행 합니다.
- (실행 초기에 한 번만 실행하면 됩니다!)

In [1]:
! pip install -r requirements.txt

- 노트북에서 사용하는 필요 Library 를 import 합니다

In [2]:
from typing import Union
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from pathlib import Path
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import SGDOneClassSVM
from sklearn.cluster import DBSCAN

- 모델 학습에 필요한 `random_seed` 를 설정하고, `DATA_PATH` 를 설정합니다.
- 해당 `DATA_PATH` 에는 `train.csv`, `test.csv` 파일이 있다고 가정합니다.

In [3]:
RANDOM_SEED = 42
DATA_PATH = Path("../data")

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


### 2. Load Train Data


In [4]:
train_data = pd.read_csv(DATA_PATH / "train.csv")

### 고민해볼 사항 🤔
- 일반적인 ML Model 에서 상관관계가 거의 1인 변수들의 경우, 중복된 정보량을 담고 있으므로 학습에 제거하는 경우가 있습니다.
- 하지만 학습 데이터인 **정상 데이터 대해 상관 관계를 계산** 한 것이고, 비정상 데이터에선 동일 데이터의 상관관계가 달라질 수 있습니다.
- 실제 화학 공정에서 이상이 발생하는 경우는 특정 변수들과의 상관 관계가 깨지는 경우도 포함될 수 있습니다.
- 따라서, 상관 관계에 따라 변수를 선택할지 제거할지 결정할 땐 **모델의 성능**과 같이 고려하는 것이 바람직합니다.

### 4. Data Process

- `simulationRun` 과 `sample` 변수는 큰 정보량을 담고 있지 않기 때문에, 공정의 센서 값을 반영하는 변수들을 선택합니다.
- 따라서, `process_data` 함수에서는 `xmeas`, `xmv` 와 같은 숫자형 변수들을(numerical value) 초도 모델 학습에 사용합니다.
- 정규화와 feature engineering의 과정은 포함되지 않았으므로, `process_data` 함수 전, 후로 더 다양한 전처리 과정을 추가해보시길 권장합니다.

In [7]:
def process_data(df) -> pd.DataFrame:
    numeric_cols = [
        'xmeas_1', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
        'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_2',
        'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25',
        'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_3', 'xmeas_30',
        'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36',
        'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_4', 'xmeas_40', 'xmeas_41',
        'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmv_1',
        'xmv_10', 'xmv_11', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6',
        'xmv_7', 'xmv_8', 'xmv_9'
    ]
    return df[numeric_cols]

In [8]:
train_X = process_data(train_data)

### 5. Model Training

In [9]:

class AnomalyDetection:
    def __init__(self, df, eps=25, min_samples=4):
        self.df = df
        self.train_X = self.process_data(df)
        self.eps = eps
        self.min_samples = min_samples
        self.dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)
        self.cluster_labels = None
    
    def process_data(self, df):
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if 'cluster' in numeric_cols:
            numeric_cols.remove('cluster')
        return df[numeric_cols]
    
    def fit_dbscan(self):
        self.dbscan_model.fit(self.train_X)
        self.cluster_labels = self.dbscan_model.labels_
        return self.dbscan_model
    
    def add_cluster_labels(self):
        self.train_X['cluster'] = self.cluster_labels
    
    def get_cluster_info(self):
        n_clusters = len(set(self.cluster_labels)) - (1 if -1 in self.cluster_labels else 0)
        n_noise = list(self.cluster_labels).count(-1)
        return n_clusters, n_noise

    def apply_to_test_data(self, test_df):
        test_X = self.process_data(test_df)
        test_dbscan = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(test_X)
        test_X['cluster'] = test_dbscan.labels_
        
        # 필터링 및 결과 저장
        non_zero_clusters = test_X[test_X['cluster'] != 0]
        test_X.to_csv('../submission/test_X.csv', index=False)

        test_X['faultNumber'] = test_X['cluster'].apply(lambda x: 0 if x == 0 else 1)
        output_df = test_X[['faultNumber']]
        output_df.reset_index(inplace=True)
        output_df.rename(columns={'index': 'sample'}, inplace=True)

        fault_number_file_path = '../submission/DBSCAN_output_new.csv'
        output_df.to_csv(fault_number_file_path, index=False)
        
        return non_zero_clusters, fault_number_file_path

# 사용 예
train_data = pd.read_csv("../data/train.csv")
test_data = pd.read_csv("../data/test.csv")

ad_model = AnomalyDetection(train_data)
ad_model.fit_dbscan()
ad_model.add_cluster_labels()
n_clusters, n_noise = ad_model.get_cluster_info()

print(f"Detected clusters: {n_clusters}")
print(f"Noise points: {n_noise}")

# 테스트 데이터에 적용
non_zero_clusters, fault_number_file_path = ad_model.apply_to_test_data(test_data)
print(f"File saved to: {fault_number_file_path}")


Detected clusters: 106
Noise points: 8135


/tmp/ipykernel_1480547/894251191.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df.rename(columns={'index': 'sample'}, inplace=True)


File saved to: ../submission/DBSCAN_output_new.csv
